In [1]:
%pylab inline
%autosave 25

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


Autosaving every 25 seconds


## Get some data

In [2]:
data_loc = '../data/FDA-COVID19_files_v1.0/'

In [3]:
store = pd.HDFStore(data_loc + 'sampled_data.h5')
df_features = pd.DataFrame(store['df' ])
store.close()
print('rows: {:,}, columns: {:,}'.format(len(df_features), len(df_features.columns)))

df_features.head()

rows: 22,172, columns: 16,391


,cid,pid,activity,AEK,VEL,EKF,LGM,VKN,LKP,NEE,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
58,204,Q99VQ4,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
97,204,EDT84149,0,3.936758,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
98,204,AAX80043,0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0
126,204,P0C6X7,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,2.367871,...,0,0,0,0,0,0,0,0,0,0
136,8549,P08659,0,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,...,0,0,0,0,0,0,0,0,0,0


## Check column variance

How many columns have zero variance? We can drop those.

In [4]:
no_var_cols = [col for col in df_features.columns if df_features[col].nunique() == 1]

len(no_var_cols)

0

In [25]:
var_cols = [col for col in df_features.columns if df_features[col].nunique() > 1]
df_novar_cols_dropped = df_features[var_cols].copy()

print('Dropped {:,} columns that have zero variance.'.format(len(df_features.columns)-len(var_cols)))

print('rows: {:,}, columns: {:,}'.format(len(df_novar_cols_dropped), len(df_novar_cols_dropped.columns)))

Dropped 685 columns that have zero variance.
rows: 22,172, columns: 16,391


## Scale the data

In [4]:
df_features.drop(columns=['cid', 'pid', 'activity'], inplace=True)

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_features.values)

In [6]:
scaled_data.T[1000].mean()

-5.127495838066752e-18

## Find highly correlated columns

In [7]:
df_scaled = pd.DataFrame(scaled_data, columns=df_features.columns)

In [8]:
df_scaled.head()

,AEK,VEL,EKF,LGM,VKN,LKP,NEE,TPN,SRL,KEY,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
1,4.806774,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,3.380128,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
2,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
3,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,4.585527,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111
4,-0.248058,-0.258672,-0.172979,-0.08229,-0.168249,-0.186884,-0.208103,-0.121681,-0.254555,-0.196883,...,-0.165629,-0.133284,-0.113502,-0.139124,-0.162146,-0.110194,-0.167203,-0.109141,-0.132405,-0.114111


In [9]:
a = [0, 1, 2, 3, 4, 5, 6]

res = [(x, y) for i,x in enumerate(a) for j,y in enumerate(a) if i > j]

res

[(1, 0),
 (2, 0),
 (2, 1),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5)]

In [10]:
b = [1,1,2,3,4,5,6]
np.corrcoef(a, b)[0][1]

0.9882117688026186

In [54]:
cols = df_scaled.columns.tolist()

out = [(x, y) for i,x in enumerate(cols) for j,y in enumerate(cols) if i > j and 
       np.corrcoef(df_scaled[x], df_scaled[y])[0][1] > 0.95]

In [55]:
out

[]

In [ ]:
df_active = df_interactions[df_interactions['activity']==1]
df_active = df_active.astype({'cid': 'category'})
df_active.head()

In [ ]:
num_active = len(df_active)
total = len(df_interactions)
print('There are {:,} active entries out of {:,} interactions ({:.2%}).'.format(num_active, total, num_active/total))

In [ ]:
print('number of unique cids is {:,}.'.format(df_active['cid'].nunique()))
print('number of unique pids is {:,}.'.format(df_active['pid'].nunique()))    

In [ ]:
import seaborn as sns; sns.set(style="white", color_codes=True)

pyplot.figure(figsize=(15, 15))
df_heatmap = df_active[:100].pivot_table(values='activity', index='cid', columns='pid', aggfunc=np.sum)
sns.heatmap(df_heatmap, annot=True)
plt.show()

In [ ]:
s_cid = df_active['cid'].value_counts()
s_pid = df_active['pid'].value_counts()

print(len(s_cid[s_cid > 10]))

print(len(s_pid[s_pid > 10]))

In [ ]:
import matplotlib.pyplot as plt

#s2 = pd.Series([1,2,3,4,5,2,3,333,2,123,434,1,2,3,1,11,11,432,3,2,4,3,3,3,54,34,24,2,223,2535334,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30000, 2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
#prob = s2.value_counts(normalize=True)
#threshold = 0.02
#mask = prob > threshold
#tail_prob = prob.loc[~mask].sum()
#prob = prob.loc[mask]
#prob['other'] = tail_prob

pyplot.figure(figsize=(20, 5))
s_cid[:100].plot(kind='bar')
plt.xticks(rotation=90)
plt.show()


In [ ]:
df_inactive = df_interactions[df_interactions['activity']==0]
df_inactive = df_inactive.astype({'cid': 'category'})
df_inactive.head()